# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [19]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [20]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\mateu\OneDrive\Documents\Insper\Segundo Semestre\Ciência dos Dados\Projeto 1\Projeto1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [21]:
filename = 'Excel-Samsung.xlsx'

In [22]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Unnamed: 1
0,@devilslawyerr_ toda vez q eu entro no tiktok ...,1
1,@amarcosta01 @qgjulietteof @seara_brasil tb am...,0
2,@prodzefyre @luagluiz @brrrb3rry um samsung e ...,3
3,é humanamente impossível comprar uma máquina l...,2
4,[magazine luiza] (cliente ouro) carregador por...,1


In [23]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Unnamed: 1
0,"@ai_cansado esmola da samsung, ainda não tô pr...",0.0
1,vsf essa propaganda da samsung com a juliettr\...,0.0
2,o bts ainda fez uma propaganda pra samsung no ...,0.0
3,aaaaaaaaaaaaa vem aí????? não brinca com meu j...,0.0
4,"eu defendia a samsung, mas agr a juliette é ga...",0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


0 - Irrelevante - Os tweets considerados irrelevantes são aqueles que não fazem menção aos produtos da Samsung

1 - Neutro - Os tweets considerados neutros são aqueles que fazem uma menção a algum produto da Samsung, porém não é algo significativo. Ex: aaa samsung eu to travada que nem o celular de vcs

2 - Relevante - Os tweets considerados relevantes são aqueles que fazem uma menção construtiva a algum produto da Samsung ou alguma funcionalidade que eles apresentam

3 - Muito Relevante - Os tweets considerados muito relevantes são aqueles que fazem uma menção construtiva de 2 ou + produtos da Samsumg ou 2 ou + funcionalidades que eles apresentam. Isso inclui venda de produtos com descrições detalhadas


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [54]:

   # for tweet in train:
   # tweet2 = ""
   # for caractere in tweet:
  #  if caractere in limpeza:
   #     caractere = caractere.replace(caractere,"")
  #  tweet2 += caractere"


#def limpeza(tweet):
   # caracteres_especiais = "\n,()[]{}#'"";:"
    #pattern = re.compile(caracteres_especiais)
   # tweet = tweet.replace(pattern,"")
   # return tweet

#Removendo pontuações
def limpeza_caracteres_iniciais(tweet):
    caracteres_especiais = '[!-.:?;_,<>/\][)(#%$|\n]' 
    pattern = re.compile(caracteres_especiais)
    substituições = re.sub(pattern,'', tweet)
    return substituições

#Removendo menções
def limpeza_menções(tweet):
    mencoes = "@[A-Za-z0-9]+"
    pattern = re.compile(mencoes)
    substituições = re.sub(pattern,'',tweet)
    return substituições

train.Treinamento = train.Treinamento.apply(limpeza_caracteres_iniciais)
train.Treinamento = train.Treinamento.apply(limpeza_menções)

train

,Treinamento,Relevancia
0,toda vez q eu entro no tiktok ta essa p no an...,1
1,tb amo miga 😍🎵💜📱mpn juliettefashion juliet...,0
2,um samsung e um xiaomi de 500 conto não val...,3
3,é humanamente impossível comprar uma máquina l...,2
4,magazine luiza cliente ouro carregador portáti...,1
...,...,...
695,nss a juli finalmente anunciando a parceria de...,0
696,samsung pq demorou tanto espero essa parceri...,0
697,amg se for pouco demais tu leva numa assistê...,1
698,não brinca com isso samsung,0


In [56]:
train = train.rename(columns={"Unnamed: 1": "Relevancia"})
train

,Treinamento,Relevancia
0,toda vez q eu entro no tiktok ta essa p no an...,1
1,tb amo miga 😍🎵💜📱mpn juliettefashion juliet...,0
2,um samsung e um xiaomi de 500 conto não val...,3
3,é humanamente impossível comprar uma máquina l...,2
4,magazine luiza cliente ouro carregador portáti...,1
...,...,...
695,nss a juli finalmente anunciando a parceria de...,0
696,samsung pq demorou tanto espero essa parceri...,0
697,amg se for pouco demais tu leva numa assistê...,1
698,não brinca com isso samsung,0


In [61]:
irrelevante = train.loc[train.Relevancia == 0, :]
neutro = train.loc[train.Relevancia == 1, :]
relevante = train.loc[train.Relevancia == 2, :]
muito_relevante = train.loc[train.Relevancia == 3, :]

In [62]:
irrelevante

,Treinamento,Relevancia
1,tb amo miga 😍🎵💜📱mpn juliettefashion juliet...,0
8,adoraria que o galiotte pegasse os 300 milh...,0
12,✅curte ✅comenta ✅compatilhar✅salve a foto da ...,0
14,hoje eu acordei daquele jeitinho do coringa so...,0
16,httpstcooigc3turnh hj foi tão bom space da ju ...,0
...,...,...
693,oi nesse link eu trago procedimentos para a o...,0
695,nss a juli finalmente anunciando a parceria de...,0
696,samsung pq demorou tanto espero essa parceri...,0
698,não brinca com isso samsung,0


In [63]:
neutro

,Treinamento,Relevancia
0,toda vez q eu entro no tiktok ta essa p no an...,1
4,magazine luiza cliente ouro carregador portáti...,1
5,no congresso mundial de conservação debates so...,1
7,as embaixadas da juliette1 globoplay 2 loccita...,1
10,eu acho que ela vai anunciar daqui uns dias nã...,1
...,...,...
681,é bem provável q a samsung tá bem fodase pra ...,1
684,samsung – linha galaxy z lança desafio tik tok...,1
687,burrice sua vc não conhece a história do car...,1
690,só eu achei esquisito a nova atualização da sa...,1


In [64]:
relevante

,Treinamento,Relevancia
3,é humanamente impossível comprar uma máquina l...,2
6,pucrio apple e samsung abrem capacitação em te...,2
11,nunca falta la promo a samsung lmao,2
13,samsung segue xiaomi e lança ram virtual em ce...,2
17,compre online eletrônicos e tecnologia encontr...,2
...,...,...
669,adicionar relogio na tela inicial do celular s...,2
678,submarino preção😱‼fone de ouvido samsung galax...,2
686,menino tu me deu uma ideia agora vo baixar no...,2
688,nossa linha j da samsung é um terror kkkkkobr...,2


In [65]:
muito_relevante

,Treinamento,Relevancia
2,um samsung e um xiaomi de 500 conto não val...,3
9,quero eu um samsung desse pow,3
22,essa cam da samsung um lixo dois pixels tá ótimo,3
33,samsung é um péssimo atendimento só indo pra j...,3
34,eu sinceramente quando joguei no seu cll ac...,3
...,...,...
655,samsung até no espaço meus amigos,3
660,alguém tá tendo dificuldade em reproduzir con...,3
661,magalu está faltando opções de smartwatchs no...,3
676,smartphone samsung galaxy s20 fe 128gb cloud n...,3


In [86]:
for i,tweet in enumerate(irrelevante.Treinamento):
    tweet = tweet.split()
    if i == 0:
        lista_irrelevante = tweet
    else:
        lista_irrelevante += tweet
    
Irrelevante = pd.Series(lista_irrelevante)
Irrelevante.value_counts()

samsung      230
a            131
o            122
da           120
de           120
            ... 
pobreeeee      1
qro            1
vídeos         1
somente        1
bateu          1
Length: 1508, dtype: int64

In [87]:
for i,tweet in enumerate(neutro.Treinamento):
    tweet = tweet.split()
    if i == 0:
        lista_neutro = tweet
    else:
        lista_neutro += tweet
    
Neutro = pd.Series(lista_neutro)
Neutro.value_counts()

samsung     177
a            91
o            81
de           78
e            75
           ... 
infernal      1
opino         1
yoon          1
mó            1
sdds          1
Length: 1232, dtype: int64

In [89]:
for i,tweet in enumerate(relevante.Treinamento):
    tweet = tweet.split()
    if i == 0:
        lista_relevante = tweet
    else:
        lista_relevante += tweet
    
Relevante = pd.Series(lista_relevante)
Relevante.value_counts()

samsung               147
de                     81
que                    79
e                      76
o                      76
                     ... 
httpstcoh2zsdgqokc      1
cria                    1
bate                    1
bordas                  1
roda                    1
Length: 1166, dtype: int64

In [91]:
for i,tweet in enumerate(muito_relevante.Treinamento):
    tweet = tweet.split()
    if i == 0:
        lista_muito_relevante = tweet
    else:
        lista_muito_relevante += tweet
    
Muito_Relevante = pd.Series(lista_muito_relevante)
Muito_Relevante.value_counts()

samsung     106
e            66
de           58
da           50
a            47
           ... 
dynamic       1
flip          1
😁             1
whatsapp      1
ps4           1
Length: 1061, dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**